# Notebook 03: The Honoré (1992) Trimmed LAD Estimator -- SOLUTIONS

**This is the worked solution notebook.**  
It provides complete, working solutions for all 4 exercises from `03_honore_estimator.ipynb`.

> Instructors: do not distribute this file to students before they complete the tutorial notebook.

## Exercise Overview

| Exercise | Topic | Difficulty | Key Concept |
|----------|-------|------------|-------------|
| 1 | Trimming Rate Analysis | Easy | How censoring threshold affects informative pairs |
| 2 | Honoré vs RE Tobit Robustness | Medium | Coefficient stability across covariate subsets |
| 3 | Sensitivity to Panel Length | Medium | Impact of T on pairwise estimation |
| 4 | Simulated Comparison — When RE Fails | Hard | Demonstrating RE bias with correlated effects |

---

## Setup

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Statistical libraries
from scipy import stats
import statsmodels.api as sm

# PanelBox imports
from panelbox.models.censored import PooledTobit, RandomEffectsTobit, HonoreTrimmedEstimator

# Visualization configuration
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

# Set random seed for reproducibility
np.random.seed(42)

# Define paths (relative to notebook location in examples/censored/solutions/)
BASE_DIR = Path('..')
DATA_DIR = BASE_DIR / 'data'
OUTPUT_DIR = BASE_DIR / 'outputs'
FIGURES_DIR = OUTPUT_DIR / 'figures' / '03_honore'
TABLES_DIR = OUTPUT_DIR / 'tables' / '03_honore'

# Create output directories
FIGURES_DIR.mkdir(parents=True, exist_ok=True)
TABLES_DIR.mkdir(parents=True, exist_ok=True)

print('Setup complete!')

---

## Data Loading

In [ ]:
# Load consumer durables panel data
df = pd.read_csv(DATA_DIR / 'consumer_durables_panel.csv')

print(f'Dataset shape: {df.shape}')
print(f'Unique households: {df["id"].nunique()}')
print(f'Time periods: {df["time"].nunique()} ({df["time"].min()}-{df["time"].max()})')
print(f'Censoring rate: {(df["spending"] == 0).mean() * 100:.1f}%')
print()
display(df.head(8))

---

## Data Preparation and Base Models

Before tackling the exercises, we prepare the estimation data and fit baseline
Honoré and RE Tobit models that will serve as references for all exercises.

In [ ]:
# Prepare base arrays
var_names_honore = ['income', 'wealth', 'household_size', 'credit_score']

# Use subsample of 50 households for Honoré (as in main notebook)
np.random.seed(42)
all_ids = df['id'].unique()
subsample_ids = np.random.choice(all_ids, size=50, replace=False)
df_sub = df[df['id'].isin(subsample_ids)].copy().reset_index(drop=True)

# Honoré arrays (NO constant)
y_honore = df_sub['spending'].values
X_honore = df_sub[var_names_honore].values
groups_honore = df_sub['id'].values
time_honore = df_sub['time'].values

# RE Tobit arrays (WITH constant)
X_re = sm.add_constant(X_honore)

print(f'Subsample: {df_sub["id"].nunique()} households, {len(df_sub)} observations')
print(f'Censoring rate: {(df_sub["spending"] == 0).mean() * 100:.1f}%')
print(f'Covariates (Honoré): {var_names_honore}')
print(f'X_honore shape: {X_honore.shape} (no constant)')
print(f'X_re shape: {X_re.shape} (with constant)')

In [ ]:
# ============================================================
# Fit base Honoré model (subsample, all covariates)
# ============================================================

print('Fitting base Honoré Trimmed LAD Estimator...')
print('=' * 60)

try:
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        honore_base = HonoreTrimmedEstimator(
            endog=y_honore,
            exog=X_honore,
            groups=groups_honore,
            time=time_honore,
            censoring_point=0.0
        )
        honore_base_result = honore_base.fit(verbose=True)
    
    print(f'\nConverged: {honore_base_result.converged}')
    print(f'Iterations: {honore_base_result.n_iter}')
    print(f'Trimmed pairs: {honore_base_result.n_trimmed}')
    print('\nCoefficients:')
    for i, name in enumerate(var_names_honore):
        print(f'  {name:>20s}: {honore_base_result.params[i]:.4f}')
    honore_base_ok = True
except Exception as e:
    print(f'Base Honoré estimation failed: {e}')
    honore_base_ok = False

In [ ]:
# ============================================================
# Fit base RE Tobit model (same subsample)
# ============================================================

print('Fitting base Random Effects Tobit...')
print('=' * 60)

try:
    re_base = RandomEffectsTobit(
        endog=y_honore,
        exog=X_re,
        groups=groups_honore,
        time=time_honore,
        censoring_point=0.0,
        censoring_type='left',
        quadrature_points=12
    )
    re_base.fit(method='BFGS', maxiter=1000)
    print(re_base.summary())
    re_base_ok = True
except Exception as e:
    print(f'Base RE Tobit estimation failed: {e}')
    re_base_ok = False

---

## Exercise 1: Trimming Rate Analysis (Easy)

**Task**: Using the consumer durables panel, calculate the trimming rate for
different censoring thresholds. Vary the censoring point from 0 to 500 in
steps of 100. Plot the relationship between censoring threshold and the
percentage of pairs trimmed. At what point does the estimator become
unreliable (>80% trimmed)?

### Background

The Honoré estimator trims pairs where **both** observations are censored
(i.e., both at or below the censoring point). A higher censoring threshold
means more observations are classified as censored, which increases the
trimming rate. When too many pairs are trimmed, the estimator uses very
little information and becomes unreliable.

The trimming rate can be computed analytically without running the full
optimization: for each entity, count pairwise differences where both
observations are at or below the censoring point.

In [ ]:
# ============================================================
# Exercise 1 Solution: Trimming Rate Analysis
# ============================================================

# Use the full dataset (N=200) for this analysis
censoring_thresholds = [0, 100, 200, 300, 400, 500]
trimming_results = []

for threshold in censoring_thresholds:
    total_pairs = 0
    trimmed_pairs = 0
    
    # Count censored observations at this threshold
    n_censored_obs = (df['spending'] <= threshold).sum()
    pct_censored_obs = 100 * n_censored_obs / len(df)
    
    for hh_id in df['id'].unique():
        hh_data = df[df['id'] == hh_id].sort_values('time')
        spending = hh_data['spending'].values
        T_hh = len(spending)
        
        for t in range(T_hh):
            for s in range(t + 1, T_hh):
                total_pairs += 1
                # Both censored at threshold?
                if spending[t] <= threshold and spending[s] <= threshold:
                    trimmed_pairs += 1
    
    kept_pairs = total_pairs - trimmed_pairs
    pct_trimmed = 100 * trimmed_pairs / total_pairs if total_pairs > 0 else 0
    
    trimming_results.append({
        'Censoring Threshold': threshold,
        'Obs Censored (%)': round(pct_censored_obs, 1),
        'Total Pairs': total_pairs,
        'Trimmed Pairs': trimmed_pairs,
        'Kept Pairs': kept_pairs,
        'Trimming Rate (%)': round(pct_trimmed, 1)
    })

trim_df = pd.DataFrame(trimming_results)

print('Trimming Rate Analysis: Varying Censoring Threshold')
print('=' * 80)
print(f'Dataset: N={df["id"].nunique()} households, T={df["time"].nunique()} periods')
print(f'Pairs per household: C(5,2) = 10')
print(f'Total pairs: {trim_df["Total Pairs"].iloc[0]}')
print()
display(trim_df)

In [ ]:
# ============================================================
# Visualization: Trimming rate vs censoring threshold
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Trimming rate vs threshold with danger zone
ax = axes[0]
ax.plot(trim_df['Censoring Threshold'], trim_df['Trimming Rate (%)'],
        'o-', markersize=10, linewidth=2.5, color='steelblue', zorder=5)

# Danger zone above 80%
ax.axhspan(80, 100, alpha=0.15, color='red', label='Danger zone (>80% trimmed)')
ax.axhline(y=80, color='red', linestyle='--', linewidth=1.5, alpha=0.7)

# Annotate data points
for _, row in trim_df.iterrows():
    ax.annotate(f'{row["Trimming Rate (%)"]:.0f}%',
                (row['Censoring Threshold'], row['Trimming Rate (%)']),
                textcoords='offset points', xytext=(0, 12),
                ha='center', fontsize=10, fontweight='bold')

ax.set_xlabel('Censoring Threshold', fontsize=12)
ax.set_ylabel('Trimming Rate (%)', fontsize=12)
ax.set_title('Trimming Rate vs Censoring Threshold', fontsize=13)
ax.set_ylim(-5, 105)
ax.legend(fontsize=10, loc='upper left')
ax.grid(alpha=0.3)

# Right: Kept pairs vs threshold
ax = axes[1]
colors = ['#D55E00' if r > 80 else '#009E73' for r in trim_df['Trimming Rate (%)']]
ax.bar(range(len(trim_df)), trim_df['Kept Pairs'], color=colors,
       alpha=0.8, edgecolor='black')
ax.set_xticks(range(len(trim_df)))
ax.set_xticklabels([f'c={t}' for t in trim_df['Censoring Threshold']])
ax.set_xlabel('Censoring Threshold', fontsize=12)
ax.set_ylabel('Number of Informative (Kept) Pairs', fontsize=12)
ax.set_title('Informative Pairs Remaining After Trimming', fontsize=13)
ax.grid(alpha=0.3, axis='y')

# Annotate bar values
for i, val in enumerate(trim_df['Kept Pairs']):
    ax.text(i, val + 15, str(val), ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'ex1_trimming_rate_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# ============================================================
# Find the threshold where trimming exceeds 80%
# ============================================================

print('Analysis: When Does the Estimator Become Unreliable?')
print('=' * 60)

danger_mask = trim_df['Trimming Rate (%)'] > 80
if danger_mask.any():
    first_danger = trim_df.loc[danger_mask].iloc[0]
    print(f'The trimming rate first exceeds 80% at threshold = {first_danger["Censoring Threshold"]}')
    print(f'  Trimming rate: {first_danger["Trimming Rate (%)"]:.1f}%')
    print(f'  Kept pairs: {first_danger["Kept Pairs"]} (out of {first_danger["Total Pairs"]})')
    print()
    print('At this point, the Honoré estimator is using very few')
    print('informative pairs and estimates become highly variable.')
else:
    print('Trimming rate stays below 80% for all tested thresholds.')
    print('The estimator remains usable across the tested range.')

print()
print('Rule of thumb:')
print('  - Trimming < 50%: Good — plenty of informative pairs')
print('  - Trimming 50-70%: Acceptable — estimates may be noisy')
print('  - Trimming 70-80%: Caution — consider alternative methods')
print('  - Trimming > 80%: Unreliable — too few informative pairs')

# Save results table
trim_df.to_csv(TABLES_DIR / 'ex1_trimming_rate_analysis.csv', index=False)
print(f'\nSaved to {TABLES_DIR / "ex1_trimming_rate_analysis.csv"}')

### Exercise 1: Discussion

**Key findings from the trimming rate analysis:**

1. **Monotonic relationship**: As the censoring threshold increases, more observations
   are classified as censored, and consequently more pairs have both members censored.
   The trimming rate increases monotonically.

2. **Original censoring point (c=0)**: At the natural censoring point (spending cannot
   be negative), the trimming rate reflects the inherent censoring in the data
   (~40% of observations are zero).

3. **Danger zone**: At higher thresholds, the trimming rate rapidly approaches 100%.
   Once above 80%, the estimator is using fewer than 20% of available pairs, making
   it highly sensitive to the specific pairs that survive trimming.

4. **Practical implication**: The Honoré estimator works best when the censoring rate
   is moderate (20-50%). With very high censoring (>70% of observations), most pairs
   are trimmed and the estimator loses efficiency dramatically. In such cases,
   the RE Tobit (despite its stronger assumptions) may be more practical.

---

## Exercise 2: Honoré vs RE Tobit Robustness (Medium)

**Task**: Estimate both Honoré and RE Tobit using only a subset of covariates:
1. `income` only
2. `income` + `household_size`
3. All four variables (`income`, `wealth`, `household_size`, `credit_score`)

For each specification, compare the `income` coefficient between the two methods
and assess whether the discrepancy is consistent across specifications.

### Motivation

If the RE assumption ($\alpha_i \perp X_{it}$) holds, both estimators should
give similar results regardless of the specification. If the RE assumption is
violated, the discrepancy may vary with the included covariates (omitted
variable bias interacts with the correlated effects).

In [ ]:
# ============================================================
# Exercise 2 Solution: Honoré vs RE Tobit Across Specifications
# ============================================================

specifications = [
    {'name': 'income only', 'vars': ['income']},
    {'name': 'income + household_size', 'vars': ['income', 'household_size']},
    {'name': 'all variables', 'vars': ['income', 'wealth', 'household_size', 'credit_score']}
]

robustness_results = []

for spec in specifications:
    spec_name = spec['name']
    spec_vars = spec['vars']
    
    print(f'\n{"=" * 60}')
    print(f'Specification: {spec_name}')
    print(f'Variables: {spec_vars}')
    print(f'{"=" * 60}')
    
    # Prepare arrays for this specification
    X_spec = df_sub[spec_vars].values
    X_spec_re = sm.add_constant(X_spec)
    
    # --- Honoré ---
    honore_income = np.nan
    honore_converged = False
    try:
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            model_h = HonoreTrimmedEstimator(
                endog=y_honore, exog=X_spec,
                groups=groups_honore, time=time_honore,
                censoring_point=0.0
            )
            result_h = model_h.fit(verbose=False)
        honore_income = result_h.params[0]  # income is always first
        honore_converged = result_h.converged
        print(f'  Honoré income coef: {honore_income:.4f} (converged: {honore_converged})')
    except Exception as e:
        print(f'  Honoré failed: {e}')
    
    # --- RE Tobit ---
    re_income = np.nan
    re_income_se = np.nan
    try:
        model_re = RandomEffectsTobit(
            endog=y_honore, exog=X_spec_re,
            groups=groups_honore, time=time_honore,
            censoring_point=0.0, censoring_type='left',
            quadrature_points=12
        )
        model_re.fit(method='BFGS', maxiter=1000)
        re_income = model_re.beta[1]  # skip constant
        re_income_se = model_re.bse[1]
        print(f'  RE Tobit income coef: {re_income:.4f} (SE: {re_income_se:.4f})')
    except Exception as e:
        print(f'  RE Tobit failed: {e}')
    
    # Compute difference
    diff = honore_income - re_income if not (np.isnan(honore_income) or np.isnan(re_income)) else np.nan
    pct_diff = 100 * diff / abs(re_income) if (not np.isnan(diff) and abs(re_income) > 1e-10) else np.nan
    
    robustness_results.append({
        'Specification': spec_name,
        'N Variables': len(spec_vars),
        'Honoré (income)': honore_income,
        'RE Tobit (income)': re_income,
        'RE Tobit SE': re_income_se,
        'Difference': diff,
        'Rel. Diff (%)': pct_diff,
        'Honoré Converged': honore_converged
    })

robust_df = pd.DataFrame(robustness_results)

print('\n\n' + '=' * 80)
print('Summary: Income Coefficient Across Specifications')
print('=' * 80)
display(robust_df)

In [ ]:
# ============================================================
# Visualization: Income coefficient comparison across specs
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Grouped bar chart of income coefficient
ax = axes[0]
x_pos = np.arange(len(robust_df))
width = 0.35

bars1 = ax.bar(x_pos - width / 2, robust_df['Honoré (income)'], width,
               label='Honoré TLAD', color='steelblue', alpha=0.8, edgecolor='black')
bars2 = ax.bar(x_pos + width / 2, robust_df['RE Tobit (income)'], width,
               label='RE Tobit', color='#D55E00', alpha=0.8, edgecolor='black')

# Add RE Tobit error bars
ax.errorbar(x_pos + width / 2, robust_df['RE Tobit (income)'],
            yerr=1.96 * robust_df['RE Tobit SE'],
            fmt='none', color='black', capsize=4, capthick=1.5)

ax.set_xticks(x_pos)
ax.set_xticklabels(['income\nonly', 'income +\nhh_size', 'all\nvariables'],
                    fontsize=10)
ax.set_ylabel('Income Coefficient', fontsize=12)
ax.set_title('Income Coefficient: Honoré vs RE Tobit\nAcross Specifications', fontsize=13)
ax.legend(fontsize=10)
ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
ax.grid(alpha=0.3, axis='y')

# Right: Relative difference
ax = axes[1]
colors = ['#D55E00' if abs(d) > 20 else '#009E73'
          for d in robust_df['Rel. Diff (%)'].fillna(0)]
ax.bar(x_pos, robust_df['Rel. Diff (%)'], color=colors, alpha=0.8,
       edgecolor='black')

for i, val in enumerate(robust_df['Rel. Diff (%)']):
    if not np.isnan(val):
        ax.text(i, val + (2 if val >= 0 else -4), f'{val:.1f}%',
                ha='center', fontsize=11, fontweight='bold')

ax.set_xticks(x_pos)
ax.set_xticklabels(['income\nonly', 'income +\nhh_size', 'all\nvariables'],
                    fontsize=10)
ax.set_ylabel('Relative Difference (%)', fontsize=12)
ax.set_title('Relative Difference (Honoré - RE Tobit)\nfor Income Coefficient', fontsize=13)
ax.axhline(y=0, color='black', linewidth=1)
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'ex2_robustness_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Save table
robust_df.to_csv(TABLES_DIR / 'ex2_robustness_comparison.csv', index=False)
print(f'Saved to {TABLES_DIR / "ex2_robustness_comparison.csv"}')

### Exercise 2: Discussion

**Key findings from the robustness analysis:**

1. **Consistency of discrepancy**: If the relative difference between Honoré and
   RE Tobit is roughly similar across all three specifications, this suggests a
   systematic difference rather than specification-dependent noise. A systematic
   discrepancy is evidence of **correlated individual effects** ($\alpha_i$ correlated
   with income).

2. **Changing discrepancy**: If the discrepancy varies substantially with the
   covariates included, it indicates that the omitted variable bias in the RE model
   is sensitive to the specification. This is consistent with the RE model absorbing
   omitted variable effects into the income coefficient when fewer controls are included.

3. **RE assumption assessment**: If both estimators give similar income coefficients
   (relative difference < 10-15%) across all specifications, the RE assumption may
   be approximately valid, and the RE Tobit can be used with more confidence.

4. **Practical recommendation**: Report both estimators. If they agree, use the RE
   Tobit (it provides standard errors). If they disagree, discuss the Honoré results
   as a robustness check and note that the RE assumption may be violated.

---

## Exercise 3: Sensitivity to Panel Length (Medium)

**Task**: Using the full dataset (T=5), estimate Honoré. Then artificially shorten
the panel:
- T=4 (drop the last period)
- T=3 (drop the last two periods)

Estimate Honoré for each panel length. How do the estimates and the number of
informative pairs change? At what T does estimation become unreliable?

### Background

The number of pairwise differences per entity is $\binom{T}{2} = T(T-1)/2$.
For T=5: 10 pairs, T=4: 6 pairs, T=3: 3 pairs, T=2: 1 pair.

With fewer pairs, the estimator has less information per entity and estimation
becomes less precise. Additionally, with shorter panels the probability that
at least one period is uncensored decreases, increasing the trimming rate.

In [ ]:
# ============================================================
# Exercise 3 Solution: Panel Length Sensitivity
# ============================================================

# Get the time periods available
all_times = sorted(df_sub['time'].unique())
print(f'Available time periods: {all_times}')

# Define panel length configurations
panel_configs = [
    {'label': 'T=5 (full)', 'periods': all_times},
    {'label': 'T=4 (drop last)', 'periods': all_times[:4]},
    {'label': 'T=3 (drop last 2)', 'periods': all_times[:3]}
]

panel_results = []

for config in panel_configs:
    label = config['label']
    periods = config['periods']
    T_eff = len(periods)
    pairs_per_entity = T_eff * (T_eff - 1) // 2
    
    print(f'\n{"=" * 60}')
    print(f'{label}')
    print(f'Periods: {periods}')
    print(f'Pairs per entity: C({T_eff},2) = {pairs_per_entity}')
    print(f'{"=" * 60}')
    
    # Filter data to selected periods
    df_t = df_sub[df_sub['time'].isin(periods)].copy().reset_index(drop=True)
    
    y_t = df_t['spending'].values
    X_t = df_t[var_names_honore].values
    g_t = df_t['id'].values
    t_t = df_t['time'].values
    
    n_obs = len(df_t)
    n_entities = df_t['id'].nunique()
    censoring_rate = (df_t['spending'] == 0).mean()
    
    print(f'Observations: {n_obs}')
    print(f'Entities: {n_entities}')
    print(f'Censoring rate: {100 * censoring_rate:.1f}%')
    
    # Estimate Honoré
    row = {
        'Panel Length': label,
        'T': T_eff,
        'Pairs/Entity': pairs_per_entity,
        'Total Pairs': n_entities * pairs_per_entity,
        'Censoring Rate (%)': round(100 * censoring_rate, 1),
        'N Obs': n_obs
    }
    
    try:
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            model_t = HonoreTrimmedEstimator(
                endog=y_t, exog=X_t,
                groups=g_t, time=t_t,
                censoring_point=0.0
            )
            result_t = model_t.fit(verbose=False)
        
        row['Converged'] = result_t.converged
        row['Trimmed Pairs'] = result_t.n_trimmed
        row['Trim Rate (%)'] = round(
            100 * result_t.n_trimmed / (n_entities * pairs_per_entity), 1
        ) if n_entities * pairs_per_entity > 0 else np.nan
        
        for i, name in enumerate(var_names_honore):
            row[name] = result_t.params[i]
        
        print(f'Converged: {result_t.converged}')
        print(f'Trimmed pairs: {result_t.n_trimmed}')
        print('Coefficients:')
        for i, name in enumerate(var_names_honore):
            print(f'  {name}: {result_t.params[i]:.4f}')
    
    except Exception as e:
        print(f'Estimation failed: {e}')
        row['Converged'] = False
        row['Trimmed Pairs'] = np.nan
        row['Trim Rate (%)'] = np.nan
        for name in var_names_honore:
            row[name] = np.nan
    
    panel_results.append(row)

panel_df = pd.DataFrame(panel_results)

print('\n\n' + '=' * 90)
print('Summary: Panel Length Sensitivity')
print('=' * 90)
display(panel_df)

In [ ]:
# ============================================================
# Visualization: Coefficient stability across panel lengths
# ============================================================

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

T_vals = panel_df['T'].values

# Left: Coefficient values for each variable
ax = axes[0]
for var_name in var_names_honore:
    if var_name in panel_df.columns:
        vals = panel_df[var_name].values
        ax.plot(T_vals, vals, 'o-', markersize=9, linewidth=2, label=var_name)

ax.set_xlabel('Panel Length (T)', fontsize=12)
ax.set_ylabel('Honoré Coefficient', fontsize=12)
ax.set_title('Coefficient Sensitivity\nto Panel Length', fontsize=13)
ax.set_xticks(T_vals)
ax.legend(fontsize=9)
ax.grid(alpha=0.3)

# Middle: Total pairs and trimmed pairs
ax = axes[1]
total_p = panel_df['Total Pairs'].values
trimmed_p = panel_df['Trimmed Pairs'].values
kept_p = total_p - trimmed_p

ax.bar(T_vals - 0.15, kept_p, 0.3, label='Kept Pairs', color='#009E73',
       alpha=0.8, edgecolor='black')
ax.bar(T_vals + 0.15, trimmed_p, 0.3, label='Trimmed Pairs', color='#D55E00',
       alpha=0.8, edgecolor='black')

ax.set_xlabel('Panel Length (T)', fontsize=12)
ax.set_ylabel('Number of Pairs', fontsize=12)
ax.set_title('Informative vs Trimmed Pairs\nby Panel Length', fontsize=13)
ax.set_xticks(T_vals)
ax.legend(fontsize=10)
ax.grid(alpha=0.3, axis='y')

# Right: Trimming rate
ax = axes[2]
trim_rates = panel_df['Trim Rate (%)'].values
colors = ['#D55E00' if r > 70 else '#E69F00' if r > 50 else '#009E73' for r in trim_rates]
ax.bar(T_vals, trim_rates, 0.5, color=colors, alpha=0.8, edgecolor='black')

for t, rate in zip(T_vals, trim_rates):
    if not np.isnan(rate):
        ax.text(t, rate + 1.5, f'{rate:.0f}%', ha='center', fontsize=11, fontweight='bold')

ax.axhline(y=80, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='Danger zone (80%)')
ax.set_xlabel('Panel Length (T)', fontsize=12)
ax.set_ylabel('Trimming Rate (%)', fontsize=12)
ax.set_title('Trimming Rate\nby Panel Length', fontsize=13)
ax.set_xticks(T_vals)
ax.set_ylim(0, 100)
ax.legend(fontsize=10)
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'ex3_panel_length_sensitivity.png', dpi=300, bbox_inches='tight')
plt.show()

# Save table
panel_df.to_csv(TABLES_DIR / 'ex3_panel_length_sensitivity.csv', index=False)
print(f'Saved to {TABLES_DIR / "ex3_panel_length_sensitivity.csv"}')

In [ ]:
# ============================================================
# Theoretical analysis: scaling of pairs with T
# ============================================================

print('Theoretical Scaling: Pairwise Differences and Panel Length')
print('=' * 60)
print(f'{"T":>4s} {"C(T,2)":>8s} {"Ratio to T=5":>14s} {"Information":>20s}')
print('-' * 50)
for T_val in [2, 3, 4, 5, 8, 10]:
    pairs = T_val * (T_val - 1) // 2
    ratio = pairs / 10  # relative to T=5
    reliability = ('Too few' if T_val <= 2 else
                   'Marginal' if T_val == 3 else
                   'Adequate' if T_val <= 5 else 'Good')
    print(f'{T_val:>4d} {pairs:>8d} {ratio:>14.1f}x {reliability:>20s}')

print()
print('With T=2, there is only 1 pair per entity — too few for reliable estimation.')
print('With T=3, there are 3 pairs, which is marginal.')
print('With T>=5, the estimator has ample pairwise information per entity.')

### Exercise 3: Discussion

**Key findings from the panel length sensitivity analysis:**

1. **Pairs grow quadratically**: The number of pairwise differences per entity
   scales as $T(T-1)/2$. Reducing from T=5 to T=3 cuts the number of pairs
   from 10 to 3 per entity — a 70% reduction in information.

2. **Trimming rate increases**: With shorter panels, each entity has fewer
   opportunities to have at least one uncensored observation in a pair.
   Entities that are censored in most periods contribute very few (or zero)
   informative pairs when the panel is short.

3. **Coefficient stability**: If coefficients remain roughly similar across
   T=3, T=4, T=5, the estimator is performing well. Large changes suggest
   instability due to insufficient information.

4. **Practical threshold**: T=3 is the minimum for the Honoré estimator
   (T=2 gives only 1 pair per entity and is too unstable). T=5 or more
   is recommended for reliable results.

---

## Exercise 4: Simulated Comparison — When RE Fails (Hard)

**Task**: Generate synthetic data where $\alpha_i$ is deliberately correlated
with $X_i$:

```python
alpha_i = 2.0 * X_mean_i + u_i
```

1. Estimate both RE Tobit and Honoré on this DGP
2. Compare with the true $\beta$ used in the simulation
3. Show that RE Tobit is biased but Honoré recovers the true $\beta$

### Background

The RE Tobit assumes $\alpha_i \perp X_{it}$. When this assumption fails
(correlated effects), the RE estimator is inconsistent — it conflates the
direct effect of $X$ on $y$ with the indirect effect through $\alpha_i$.
The Honoré estimator, by differencing out $\alpha_i$, is immune to this problem.

In [ ]:
# ============================================================
# Exercise 4 Solution: Simulated DGP with Correlated Effects
# ============================================================

# DGP parameters
N_sim = 100       # entities
T_sim = 5         # periods
beta_true = np.array([1.5, 0.8])  # true coefficients for x1, x2
sigma_eps = 2.0   # error std dev
alpha_corr = 2.0  # correlation strength: alpha_i = alpha_corr * x_mean_i + u_i
sigma_u = 1.5     # std dev of the uncorrelated part of alpha
censoring_point = 0.0

np.random.seed(123)

# Generate data
sim_data = []

for i in range(N_sim):
    # Generate covariates
    x1_i = np.random.normal(3.0, 1.5, size=T_sim)
    x2_i = np.random.normal(1.0, 0.5, size=T_sim)
    
    # Individual mean of x1 (used to create correlated alpha)
    x1_mean_i = x1_i.mean()
    
    # Correlated individual effect: alpha_i depends on x1_mean
    u_i = np.random.normal(0, sigma_u)
    alpha_i = alpha_corr * x1_mean_i + u_i
    
    # Generate outcomes
    for t in range(T_sim):
        eps_it = np.random.normal(0, sigma_eps)
        y_star = beta_true[0] * x1_i[t] + beta_true[1] * x2_i[t] + alpha_i + eps_it
        y_obs = max(censoring_point, y_star)  # left-censoring
        
        sim_data.append({
            'id': i,
            'time': t + 1,
            'y': y_obs,
            'x1': x1_i[t],
            'x2': x2_i[t],
            'alpha_true': alpha_i
        })

df_sim = pd.DataFrame(sim_data)

print('Simulated Data (Correlated Fixed Effects DGP)')
print('=' * 60)
print(f'True beta = {beta_true}')
print(f'Correlation: alpha_i = {alpha_corr} * mean(x1_i) + u_i')
print(f'sigma_eps = {sigma_eps}, sigma_u = {sigma_u}')
print(f'N = {N_sim}, T = {T_sim}')
print(f'Censoring rate: {(df_sim["y"] == censoring_point).mean() * 100:.1f}%')
print(f'Mean alpha: {df_sim.groupby("id")["alpha_true"].first().mean():.2f}')
print()

# Verify correlation between alpha and X
entity_means = df_sim.groupby('id').agg(
    x1_mean=('x1', 'mean'),
    alpha=('alpha_true', 'first')
)
corr_alpha_x = entity_means['x1_mean'].corr(entity_means['alpha'])
print(f'Corr(alpha_i, mean(x1_i)) = {corr_alpha_x:.3f}')
print('This high correlation means the RE assumption is VIOLATED.')

In [ ]:
# ============================================================
# Estimate RE Tobit and Honoré on simulated data
# ============================================================

y_sim = df_sim['y'].values
X_sim = df_sim[['x1', 'x2']].values
X_sim_re = sm.add_constant(X_sim)
groups_sim = df_sim['id'].values
time_sim = df_sim['time'].values

# --- RE Tobit ---
print('Estimating RE Tobit on simulated data...')
print('=' * 60)

re_sim_beta = [np.nan, np.nan]
re_sim_se = [np.nan, np.nan]
try:
    re_sim = RandomEffectsTobit(
        endog=y_sim, exog=X_sim_re,
        groups=groups_sim, time=time_sim,
        censoring_point=censoring_point,
        censoring_type='left',
        quadrature_points=12
    )
    re_sim.fit(method='BFGS', maxiter=1000)
    re_sim_beta = re_sim.beta[1:3]  # skip constant
    re_sim_se = re_sim.bse[1:3]
    print(f'  x1 coef: {re_sim_beta[0]:.4f} (SE: {re_sim_se[0]:.4f})')
    print(f'  x2 coef: {re_sim_beta[1]:.4f} (SE: {re_sim_se[1]:.4f})')
    re_sim_ok = True
except Exception as e:
    print(f'RE Tobit failed: {e}')
    re_sim_ok = False

# --- Honoré ---
# Use subsample for computational feasibility
print(f'\nEstimating Honoré on simulated data (N={N_sim})...')
print('=' * 60)

honore_sim_beta = [np.nan, np.nan]
try:
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        honore_sim = HonoreTrimmedEstimator(
            endog=y_sim, exog=X_sim,
            groups=groups_sim, time=time_sim,
            censoring_point=censoring_point
        )
        honore_sim_result = honore_sim.fit(verbose=False)
    
    honore_sim_beta = honore_sim_result.params
    print(f'  x1 coef: {honore_sim_beta[0]:.4f}')
    print(f'  x2 coef: {honore_sim_beta[1]:.4f}')
    print(f'  Converged: {honore_sim_result.converged}')
    print(f'  Trimmed pairs: {honore_sim_result.n_trimmed}')
    honore_sim_ok = True
except Exception as e:
    print(f'Honoré failed: {e}')
    honore_sim_ok = False

In [ ]:
# ============================================================
# Compare: True beta vs RE Tobit vs Honoré
# ============================================================

var_names_sim = ['x1', 'x2']

sim_comparison = pd.DataFrame({
    'Variable': var_names_sim,
    'True beta': beta_true,
    'RE Tobit': re_sim_beta,
    'Honoré TLAD': honore_sim_beta
})

sim_comparison['RE Bias'] = sim_comparison['RE Tobit'] - sim_comparison['True beta']
sim_comparison['RE Bias (%)'] = 100 * sim_comparison['RE Bias'] / sim_comparison['True beta'].abs()
sim_comparison['Honoré Bias'] = sim_comparison['Honoré TLAD'] - sim_comparison['True beta']
sim_comparison['Honoré Bias (%)'] = 100 * sim_comparison['Honoré Bias'] / sim_comparison['True beta'].abs()

print('Simulated DGP: Comparison of Estimators')
print('=' * 80)
print(f'DGP: alpha_i = {alpha_corr} * mean(x1_i) + u_i')
print(f'RE assumption VIOLATED: Corr(alpha, x1_mean) = {corr_alpha_x:.3f}')
print()
display(sim_comparison.round(4))

print('\nInterpretation:')
print('-' * 60)
print('  RE Tobit is BIASED because it ignores the correlation between')
print('  alpha_i and x1. The RE estimate of beta_x1 conflates the direct')
print('  effect of x1 and the indirect effect through alpha_i.')
print()
print('  Honoré eliminates alpha_i through pairwise differencing and')
print('  should recover estimates closer to the true beta values.')

In [ ]:
# ============================================================
# Visualization: True beta vs RE Tobit vs Honoré
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Bar chart of coefficient estimates
ax = axes[0]
x_pos = np.arange(len(var_names_sim))
width = 0.25

bars_true = ax.bar(x_pos - width, beta_true, width,
                    label='True $\\beta$', color='#009E73', alpha=0.9, edgecolor='black')
bars_re = ax.bar(x_pos, re_sim_beta, width,
                  label='RE Tobit', color='#D55E00', alpha=0.8, edgecolor='black')
bars_hon = ax.bar(x_pos + width, honore_sim_beta, width,
                   label='Honoré TLAD', color='steelblue', alpha=0.8, edgecolor='black')

# Add RE Tobit error bars if available
if re_sim_ok:
    ax.errorbar(x_pos, re_sim_beta, yerr=1.96 * np.array(re_sim_se),
                fmt='none', color='black', capsize=4, capthick=1.5)

ax.set_xticks(x_pos)
ax.set_xticklabels(var_names_sim, fontsize=12)
ax.set_ylabel('Coefficient Estimate', fontsize=12)
ax.set_title('True $\\beta$ vs Estimates\n(Correlated Effects DGP)', fontsize=13)
ax.legend(fontsize=10)
ax.axhline(y=0, color='gray', linewidth=0.5)
ax.grid(alpha=0.3, axis='y')

# Right: Bias comparison
ax = axes[1]
re_bias_pct = sim_comparison['RE Bias (%)'].values
hon_bias_pct = sim_comparison['Honoré Bias (%)'].values

x_pos2 = np.arange(len(var_names_sim))
width2 = 0.35

bars_re_b = ax.bar(x_pos2 - width2 / 2, re_bias_pct, width2,
                    label='RE Tobit Bias', color='#D55E00', alpha=0.8, edgecolor='black')
bars_hon_b = ax.bar(x_pos2 + width2 / 2, hon_bias_pct, width2,
                     label='Honoré Bias', color='steelblue', alpha=0.8, edgecolor='black')

# Annotate bias values
for i in range(len(var_names_sim)):
    val_re = re_bias_pct[i]
    val_hon = hon_bias_pct[i]
    if not np.isnan(val_re):
        offset_re = 2 if val_re >= 0 else -4
        ax.text(i - width2 / 2, val_re + offset_re, f'{val_re:.1f}%',
                ha='center', fontsize=10, fontweight='bold')
    if not np.isnan(val_hon):
        offset_hon = 2 if val_hon >= 0 else -4
        ax.text(i + width2 / 2, val_hon + offset_hon, f'{val_hon:.1f}%',
                ha='center', fontsize=10, fontweight='bold')

ax.axhline(y=0, color='black', linewidth=1)
ax.set_xticks(x_pos2)
ax.set_xticklabels(var_names_sim, fontsize=12)
ax.set_ylabel('Relative Bias (%)', fontsize=12)
ax.set_title('Bias Comparison\n(Positive = Overestimate)', fontsize=13)
ax.legend(fontsize=10)
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'ex4_simulated_bias_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Save comparison table
sim_comparison.to_csv(TABLES_DIR / 'ex4_simulated_comparison.csv', index=False)
print(f'Saved to {TABLES_DIR / "ex4_simulated_comparison.csv"}')

In [ ]:
# ============================================================
# Additional: Visualize the correlated effects
# ============================================================

fig, ax = plt.subplots(figsize=(8, 6))

ax.scatter(entity_means['x1_mean'], entity_means['alpha'],
           alpha=0.6, s=40, color='steelblue', edgecolor='gray', linewidth=0.5)

# Fit and plot regression line
slope, intercept, r_value, p_value, se = stats.linregress(
    entity_means['x1_mean'], entity_means['alpha']
)
x_line = np.linspace(entity_means['x1_mean'].min(), entity_means['x1_mean'].max(), 100)
ax.plot(x_line, slope * x_line + intercept, 'r-', linewidth=2,
        label=f'Fitted: slope={slope:.2f} (true={alpha_corr:.1f})\nr={r_value:.3f}')

ax.set_xlabel('Mean $x_{1i}$ (individual mean of covariate)', fontsize=12)
ax.set_ylabel('$\\alpha_i$ (individual fixed effect)', fontsize=12)
ax.set_title('Correlated Effects: $\\alpha_i$ vs Mean($x_{1i}$)\n'
             f'DGP: $\\alpha_i = {alpha_corr} \\cdot \\bar{{x}}_{{1i}} + u_i$', fontsize=13)
ax.legend(fontsize=11)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'ex4_correlated_effects.png', dpi=300, bbox_inches='tight')
plt.show()

print('The strong positive correlation between alpha_i and x1_mean')
print('violates the RE assumption and biases the RE Tobit upward for x1.')

In [ ]:
# ============================================================
# Explain the direction of bias
# ============================================================

print('Bias Direction Analysis')
print('=' * 60)
print()
print('DGP: y*_it = beta1 * x1_it + beta2 * x2_it + alpha_i + eps_it')
print(f'     alpha_i = {alpha_corr} * mean(x1_i) + u_i')
print()
print('The RE Tobit ignores the correlation Corr(alpha_i, x1_it) > 0.')
print('When alpha_i is positively correlated with x1:')
print('  - High x1 entities also have high alpha_i')
print('  - The RE model attributes part of alpha_i\'s effect to x1')
print('  - Result: RE overestimates beta_x1 (positive bias)')
print()
if re_sim_ok:
    print(f'Observed: RE beta_x1 = {re_sim_beta[0]:.4f} vs True = {beta_true[0]:.4f}')
    bias_direction = 'UPWARD' if re_sim_beta[0] > beta_true[0] else 'DOWNWARD'
    print(f'  --> {bias_direction} bias, as expected.')
print()
print('The Honoré estimator eliminates alpha_i through differencing,')
print('so it should be unaffected by this correlation.')
if honore_sim_ok:
    print(f'Observed: Honoré beta_x1 = {honore_sim_beta[0]:.4f} vs True = {beta_true[0]:.4f}')
    hon_bias = abs(honore_sim_beta[0] - beta_true[0])
    re_bias = abs(re_sim_beta[0] - beta_true[0]) if re_sim_ok else np.nan
    print(f'  --> Honoré |bias| = {hon_bias:.4f} vs RE |bias| = {re_bias:.4f}')

### Exercise 4: Discussion

**Key findings from the simulation:**

1. **RE Tobit bias**: The RE Tobit systematically overestimates $\beta_{x_1}$
   because it cannot separate the direct effect of $x_1$ from its indirect
   effect through $\alpha_i$ (which is positively correlated with $x_1$).
   The bias is substantial and in the expected direction (upward for x1).

2. **Honoré consistency**: The Honoré estimator eliminates $\alpha_i$ through
   pairwise differencing and produces estimates much closer to the true $\beta$.
   Any remaining difference is due to finite-sample variability, not systematic
   bias.

3. **x2 coefficient**: Since $\alpha_i$ is correlated with $x_1$ but not with
   $x_2$, the RE bias for $x_2$ should be smaller. This is a useful diagnostic:
   if RE and Honoré agree on $x_2$ but not on $x_1$, it suggests that the RE
   violation specifically involves $x_1$.

4. **Policy implication**: In empirical work, we rarely know whether $\alpha_i$
   is correlated with covariates. The Honoré estimator provides a valuable
   robustness check. If Honoré and RE Tobit estimates differ substantially,
   the RE assumption is suspect and the Honoré results should be preferred
   (or at least reported alongside the RE results).

5. **Cost of robustness**: The Honoré estimator is less efficient (higher
   variance) and does not provide standard errors without bootstrap. This
   is the price of relaxing the RE assumption.

---

## Summary

In this solution notebook, we worked through four exercises that deepened our
understanding of the Honoré (1992) Trimmed LAD estimator:

| Exercise | Key Takeaway |
|----------|-------------|
| 1. Trimming Rate | Higher censoring thresholds dramatically increase trimming; above ~80% trimmed, estimates are unreliable |
| 2. Robustness | Honoré vs RE Tobit discrepancy should be assessed across multiple specifications |
| 3. Panel Length | Pairwise information grows quadratically with T; T=3 is marginal, T=5+ recommended |
| 4. Simulated Bias | When RE assumption fails, RE Tobit is biased; Honoré recovers true parameters |

### Key Methods Used

```python
# Honoré Trimmed LAD (NO constant in exog)
from panelbox.models.censored import HonoreTrimmedEstimator
model = HonoreTrimmedEstimator(endog=y, exog=X, groups=g, time=t, censoring_point=0.0)
result = model.fit(verbose=False)

# RE Tobit (WITH constant in exog)
from panelbox.models.censored import RandomEffectsTobit
model = RandomEffectsTobit(endog=y, exog=X_const, groups=g, time=t,
                           censoring_point=0.0, censoring_type='left', quadrature_points=12)
model.fit(method='BFGS', maxiter=1000)
```

### Practical Recommendations

1. **Start with RE Tobit** for speed and standard errors
2. **Run Honoré as a robustness check** if you suspect correlated effects
3. **Compare coefficients**: if similar, RE assumption is plausible
4. **If they disagree**: report both and discuss which assumptions are more credible
5. **Check trimming rate**: if >70-80%, Honoré may be unreliable
6. **Ensure T >= 3**: T=2 is insufficient for pairwise estimation